**XGBoost Model**

In [1]:

import os
print(os.listdir("../input"))
os.cpu_count()


['trainpredict', 'testpredict']


4

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
X = pd.read_csv("../input/trainpredict/train.csv")
X_1 = pd.read_csv("../input/testpredict/test.csv")

In [4]:
train = X
test = X_1
print(train.isnull().sum(axis=0))
print(test.isnull().sum(axis = 0))

ID                   0
datetime             0
siteid         1212515
offerid              0
category             0
merchant             0
countrycode          0
browserid       608327
devid          1820299
click                0
dtype: int64
ID                  0
datetime            0
siteid         370776
offerid             0
category            0
merchant            0
countrycode         0
browserid      221906
devid          704619
dtype: int64


**Data  Pre Processing**
1. Add features of week day and hour
2. Update hour as range. Here, 24 hour has been divided into 6 parts
3. Label encode all features
4. For the missing values keep a separate label as -1


In [5]:
train['datetime'] = pd.to_datetime(train['datetime'])
train['day'] = train['datetime'].dt.weekday
train['hour'] = train['datetime'].dt.hour

test['datetime'] = pd.to_datetime(test['datetime'])
test['day'] = test['datetime'].dt.weekday
test['hour'] = test['datetime'].dt.hour

train['hour1'] = train['hour']%4
test['hour1'] = test['hour']%4

train = train.drop(columns=['ID', 'datetime', 'hour',])
test = test.drop(columns=['datetime', 'hour', ])

train.head()

,siteid,offerid,category,merchant,countrycode,browserid,devid,click,day,hour1
0,4709696.0,887235,17714,20301556,e,Firefox,NaN,0,5,1
1,5189467.0,178235,21407,9434818,b,Mozilla Firefox,Desktop,0,2,1
2,98480.0,518539,25085,2050923,a,Edge,NaN,0,2,0
3,8896401.0,390352,40339,72089744,c,Firefox,Mobile,0,1,2
4,5635120.0,472937,12052,39507200,d,Mozilla Firefox,Desktop,0,5,0


In [6]:
from sklearn.preprocessing import LabelEncoder
train = train.astype('object')
test = test.astype('object')
col_with_null = ['siteid', 'browserid','devid']
cols = ['offerid', 'category', 'merchant', 'countrycode']
for col in col_with_null:
    lbl = LabelEncoder()
    temp_train = train[col]
    temp_test = test[col]
    mask = train[col].isnull()
    mask1 = test[col].isnull()
    train[col] = train[col].astype(str)
    test[col] = test[col].astype(str)
    lbl.fit(list(train[col].values) + list(test[col].values))
    train[col] = lbl.transform(list(train[col].values))
    test[col] = lbl.transform(list(test[col].values))
    train[col] = train[col].where(~mask, temp_train)
    test[col] = test[col].where(~mask1, temp_test)
    
for col in cols:
    lbl = LabelEncoder()
    lbl.fit(list(train[col].values) + list(test[col].values))
    train[col] = lbl.transform(list(train[col].values))
    test[col] = lbl.transform(list(test[col].values))
    

In [7]:
train = train.replace('nan', np.NAN)
test = test.replace('nan',np.NAN)
train['siteid'].fillna(-1, inplace = True)
test['siteid'].fillna(-1, inplace = True)
train['browserid'].fillna(-1, inplace = True)
test['browserid'].fillna(-1, inplace = True)
train['devid'].fillna(-1, inplace = True)
test['devid'].fillna(-1, inplace = True)
train.head()

,siteid,offerid,category,merchant,countrycode,browserid,devid,click,day,hour1
0,112743.0,784773,48,127,4,2.0,-1.0,0,5,1
1,127360.0,157563,59,65,1,8.0,0.0,0,2,1
2,269055.0,458279,69,15,0,1.0,-1.0,0,2,0
3,240075.0,345067,117,507,2,2.0,1.0,0,1,2
4,140991.0,417948,36,276,3,8.0,0.0,0,5,0


In [8]:
print(test.head())
print(train.head())

        ID    siteid  offerid  category  ...    browserid  devid  day  hour1
0  IDFDJVI  199198.0   715248        48  ...          8.0    0.0    6      1
1  IDNWkTQ   82416.0   247936        36  ...          1.0    2.0    6      3
2  ID9pRmM  102642.0   823274        91  ...          8.0   -1.0    5      2
3  IDHaQaj   22971.0   541204        33  ...          1.0    2.0    6      2
4  IDT2CrF  130703.0   463364       130  ...          7.0   -1.0    6      1

[5 rows x 10 columns]
     siteid  offerid  category  merchant  ...    devid  click  day  hour1
0  112743.0   784773        48       127  ...     -1.0      0    5      1
1  127360.0   157563        59        65  ...      0.0      0    2      1
2  269055.0   458279        69        15  ...     -1.0      0    2      0
3  240075.0   345067       117       507  ...      1.0      0    1      2
4  140991.0   417948        36       276  ...      0.0      0    5      0

[5 rows x 10 columns]


In [9]:
train = train.astype(int)
for col in test.columns[1:]: test[col] = test[col].astype(int)


In [10]:
from sklearn.model_selection import train_test_split

Y_t = train['click'].astype('float')
X_t = train.drop(columns=['click'])
X_train, X_test, y_train, y_test = train_test_split(X_t, Y_t, test_size=0.25, random_state=69)

In [ ]:
import xgboost as xgb
num_round = 1000
param = {'objective': 'binary:logistic',  
         'tree_method': 'hist', 
         'max_depth': 10,
         'eta': 0.2,
         'eval_metric': 'auc',
         'early_stopping_rounds':50,
         'nthread': 4,
         'max_bin':512,
         #'grow_policy':'lossguide',
         'random_state':69,
         }
dtrain = xgb.DMatrix(X_train, label=y_train, missing = -1)
dtest = xgb.DMatrix(X_test, label=y_test, missing = -1)
gbm = xgb.train(param, dtrain, num_round, evals=[(dtest, 'test')])
predicted = gbm.predict(xgb.DMatrix(X_test, missing = -1))

In [12]:
from sklearn.metrics import confusion_matrix, roc_auc_score, accuracy_score as auc 
predicted = gbm.predict(xgb.DMatrix(X_test, missing = -1))
print(confusion_matrix(y_test, np.round(predicted)))
print(roc_auc_score(y_test, np.round(predicted)))
print(auc(y_test, np.round(predicted)))


[[2906408   18832]
 [  41522   67691]]
0.8066847017209796
0.980110418582855


In [13]:

preds = gbm.predict(xgb.DMatrix(test[test.columns[1:]], missing = -1))
sub = pd.DataFrame({'ID':test['ID'], 'click':preds})
sub.to_csv('submission_xg.csv', index=False)